In [50]:
import pandas as pd
import numpy as np
import requests
import json
import time
import os 
import gzip
import timeit
import scispacy
import spacy
from loguru import logger
from sklearn.manifold import TSNE
from scripts.vectology_functions import embed_text, create_aaa_distances

tSNE=TSNE(n_components=2)

cci_df = pd.read_csv('data/CCI-ICD10CM-v2021-1/CCI_ICD10CM_v2021-1.csv',skiprows=2)
cci_df.rename(columns={"'ICD-10-CM CODE DESCRIPTION'":"ICD-10-CM CODE DESCRIPTION","'CHRONIC INDICATOR'":"CHRONIC INDICATOR"},inplace=True)
cci_df['CHRONIC INDICATOR'] = df['CHRONIC INDICATOR'].str.replace("'","")

output='output/disease-cat

In [47]:
%%time

def make_vectors():
    f = f'{output}/vectors.pickle.gz'
    if os.path.exists(f):
        logger.info(f'{f} exists')
        res_df = pd.read_pickle(f)
    else:
        logger.info(f'\n{cci_df.head()}')
        logger.info(cci_df.shape)

        results=[]
        query_list = list(cci_df['ICD-10-CM CODE DESCRIPTION'])
        indicator_list = list(cci_df['CHRONIC INDICATOR'])
        #logger.info(queries)
        chunk=20
        for i in range(0,len(query_list),chunk):
            if i % 1000 == 0:
                print(i)
            batch = query_list[i:i+chunk]
            #print('\n',i,queries[i:i+chunk])
            res = embed_text(textList=batch,model='BioSentVec')
            for r in res:
                results.append({'name':query_list[i],'indicator':indicator_list[i],'vector':r})
        logger.info(len(results))
        res_df = pd.DataFrame(results)
        res_df.to_pickle(f)
    return res_df
res_df = make_vectors()

In [51]:
%%time

logger.info(len(res_df))
aaa = create_aaa_distances(list(res_df['vector']))
np.save(f'{output}/vectors-aaa.npy',aaa)

2021-04-12 21:12:46.722 | INFO     | __main__:<module>:1 - 73205
Creating distances...
73205


ValueError: A 2-dimensional array must be passed.

In [ ]:
%%time

# tSNE

d = pd.DataFrame(aaa)
#print(d.head())
d = d.fillna(1)
tSNE_result=tSNE.fit_transform(d)
x=tSNE_result[:,0]
y=tSNE_result[:,1]
ebi_df['x']=x
ebi_df['y']=y
#print(ebi_df.head())
plt.figure(figsize=(16,7))
sns.scatterplot(x='x',y='y',data=cci_df, legend="full", hue="MAPPING_TYPE")
plt.savefig(f'{output}/tsne.pdf')